# Introdução

# Materiais e Métodos:

- O objetivo inicial da pesquisa foi encontrar uma base de dados adequada para desenvolver um sistema de recomendação. Para isso, foi escolhida a base de dados "Book-Crossing Dataset", disponível na plataforma Kaggle, que contém informações sobre 271.379 usuários, 278.859 livros e suas avaliações, sendo que nem todos os usuários avaliaram todos os livros. 
- O sistema foi desenvolvido em Python, utilizando as bibliotecas Pandas, Scikit-learn e Kaggle, com o código sendo implementado no Jupyter Notebook. A recomendação foi realizada por meio de filtragem colaborativa com o algoritmo K-Nearest Neighbors (KNN), baseado nas classificações dos usuários.

## Dependências

In [ ]:
%pip install kagglehub
%pip install pandas
%pip install scikit-learn
%pip install matplotlib

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## DataSet

### Importando o DataSet

In [3]:
books = pd.read_csv("/home/cscarabelotti/Desktop/projects/sistemas-recomendacao/datasets/Books.csv", sep=";", header=0)
users = pd.read_csv("/home/cscarabelotti/Desktop/projects/sistemas-recomendacao/datasets/Users.csv", sep=";", low_memory=False, header=0)
ratings = pd.read_csv("/home/cscarabelotti/Desktop/projects/sistemas-recomendacao/datasets/Ratings.csv", sep=";", low_memory=False, header=0)

### Visualização dos dados importados

In [4]:
print("Books")
print(books.head(), "\n")

print("Informações sobre os livros")
print(books.info(), "\n")

Books
         ISBN                                              Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 Author  Year                Publisher  
0    Mark P. O. Morford  2002  Oxford University Press  
1  Richard Bruce Wright  2001    HarperFlamingo Canada  
2          Carlo D'Este  1991          HarperPerennial  
3      Gina Bari Kolata  1999     Farrar Straus Giroux  
4       E. J. W. Barber  1999   W. W. Norton & Company   

Informações sobre os livros
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   ISBN       271379 

In [5]:
print("Ratings")
print(ratings.head(), "\n")

print("Informações sobre os ratings")
print(ratings.info(), "\n")



Ratings
   User-ID        ISBN  Rating
0   276725  034545104X       0
1   276726  0155061224       5
2   276727  0446520802       0
3   276729  052165615X       3
4   276729  0521795028       6 

Informações sobre os ratings
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   User-ID  1149780 non-null  int64 
 1   ISBN     1149780 non-null  object
 2   Rating   1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
None 



In [6]:
print("Users")
print(users.head(), "\n")

print("Informações sobre os usuários")
print(users.info(), "\n")

Users
  User-ID  Age
0       1  NaN
1       2   18
2       3  NaN
3       4   17
4       5  NaN 

Informações sobre os usuários
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278859 entries, 0 to 278858
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   User-ID  278859 non-null  object
 1   Age      168627 non-null  object
dtypes: object(2)
memory usage: 4.3+ MB
None 



### Ajustando os dados

- A base de dados é composta por três arquivos: books.csv, ratings.csv e users.csv. Em todos os arquivos, foram removidos valores duplicados, campos vazios e espaços extras nos dados de texto.

In [ ]:
# Books
books.dropna(subset=["Title", "Author", "Publisher"], inplace=True)
books["ISBN"] = books["ISBN"].str.strip()
books["Title"] = books["Title"].str.strip()
books["Author"] = books["Author"].str.strip()
books["Publisher"] = books["Publisher"].str.strip()
books.drop_duplicates(subset=["ISBN"], inplace=True)

books.info()

- Em ratings, as avaliações duplicadas de um mesmo usuário para o mesmo livro foram eliminadas, mantendo a média das avaliações, e garantiu-se que as avaliações estivessem entre 0 e 10. 

In [ ]:
# Ratings
ratings.dropna(inplace=True)
ratings["User-ID"] = ratings["User-ID"].astype(str).str.strip()
ratings["ISBN"] = ratings["ISBN"].str.strip()
ratings = ratings[(ratings["Rating"] >= 0) & (ratings["Rating"] <= 10)]
ratings = ratings.groupby(["User-ID", "ISBN"], as_index=False)["Rating"].mean()
ratings.info()

- Em users, as idades dos usuários foram convertidas para números, validadas entre 0 e 100, e dados inválidos foram descartados (como NaN).

In [9]:
# Users
users["Age"] = pd.to_numeric(users["Age"], errors="coerce")
users = users[(users["Age"] >= 0) & (users["Age"] <= 100)]
users.drop_duplicates(inplace=True)

### Separando os dados em treino/teste

In [10]:
sampled_ratings = ratings.sample(frac=0.02, random_state=42)
train, test = train_test_split(sampled_ratings, test_size=0.2, random_state=42)
print(f"Tamanho do treino: {len(train)}")
print(f"Tamanho do teste: {len(test)}")

Tamanho do treino: 18396
Tamanho do teste: 4600


### Criando a matriz de iteração

- É criada uma matriz de iteração, que relaciona usuários (identificados por ID) com livros (identificados pelo ISBN) e suas respectivas avaliações.

- Nessa matriz, os livros são representados nas colunas, os usuários nas linhas, e as células contêm as avaliações atribuídas pelos usuários aos livros.

In [11]:
rating_matrix = sampled_ratings.pivot(index='User-ID', columns='ISBN', values='Rating')

# Implementação do Algoritmo

### Calculando a distância euclidiana

In [27]:
rating_matrix_filled = rating_matrix.fillna(0)
user_similarity = cosine_similarity(rating_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=rating_matrix_filled.index, columns=rating_matrix_filled.index)

### Realizando recomendações

- Após calcular a distância euclidiana entre os vizinhos, o algoritmo identifica os 10 usuários mais semelhantes ao usuário alvo. 

- Em seguida, seleciona os livros avaliados pelo usuário alvo e cria um dicionário vazio para armazenar as recomendações. 

- O algoritmo então verifica os livros avaliados pelos usuários vizinhos, adicionando ao dicionário aqueles que o usuário alvo ainda não avaliou. 

- Após concluir essa etapa, calcula-se a média das avaliações de cada livro recomendado, e a lista de recomendações é ordenada com base nessas médias, retornando os livros recomendados ao usuário alvo.

In [ ]:
def recommend_books(user_id, rating_matrix, user_similarity_df, books, top_n=5):

    similar_users = user_similarity_df[user_id].drop(user_id).sort_values(ascending=False).head(10)
    
    books_rated_by_user = rating_matrix.loc[user_id].dropna().index
    
    book_recommendations = {}

    for similar_user_id in similar_users.index:
        books_rated_by_similar_user = rating_matrix.loc[similar_user_id].dropna()
        
        for book, rating in books_rated_by_similar_user.items():
            if pd.isna(rating_matrix.loc[user_id, book]):
                if book not in book_recommendations:
                    book_recommendations[book] = []
                book_recommendations[book].append(rating)
    
    average_ratings = {book: np.mean(ratings) for book, ratings in book_recommendations.items()}
    
    recommended_books = sorted(average_ratings.items(), key=lambda x: x[1], reverse=True)
    
    recommended_books_with_titles = []
    for isbn, _ in recommended_books[:top_n]:
        book_title = books[books['ISBN'] == isbn]['Title'].values[0] 
        recommended_books_with_titles.append(book_title)
    
    return recommended_books_with_titles

recommended_books_list = recommend_books("100004", rating_matrix, user_similarity_df, books, top_n=5)
recommended_books_list 

['Wanderlust (Dragonlance: The Meetings Sextet, Vol. 2)',
 'The Midnight Cafe (Anita Blake, Vampire Hunter)',
 'Death of a Hussy',
 'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
 'The Mists of Avalon']

# Avaliação do Algoritmo - Resultados e Conclusão

- O sistema de recomendação foi desenvolvido utilizando o algoritmo KNN e filtragem colaborativa, baseado na base de dados “Book-Crossing Dataset”. 

- Durante o desenvolvimento, um desafio foi o alto consumo de recursos, tornando o processamento lento e custoso. 

- Além disso, a ausência de avaliações reais nos dados de treino e teste impediu uma análise mais completa, com a medição de acurácia ou precisão, por exemplo. 

- Apesar dessas limitações, a implementação do KNN foi bem-sucedida, criando um sistema básico de recomendação. 

- O projeto destacou a importância de um bom tratamento de dados para o bom funcionamento do modelo.

- O tratamento adequado dos dados, como a remoção de duplicatas e transformação da idade dos usuários, contribuiu para o sucesso do funcionamento do modelo. 

- Para trabalhos futuros, uma possibilidade de melhorar o desenvolvimento atual é trazendo o foco para a coleta de dados de usuários reais, afim de que seja possível mensurar a acurácia e eficácia das recomendações geradas pelo sistema.